In [1]:
from pathlib import Path

import requests

Path management

In [2]:
!pwd

/home/jobquiroz/full_stack_deep_learning/lab01/notebooks


In [3]:
# Fixing path
import os

os.getcwd()   # Verify where it is right now...

'/home/jobquiroz/full_stack_deep_learning/lab01/notebooks'

In [4]:
# Solution, go to lab directory:
os.chdir('/home/jobquiroz/full_stack_deep_learning/lab01/')

Download MNIST dataset

In [5]:
def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"
    
    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename

data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

Decompressing, deserialization and reading.

In [6]:
import gzip
import pickle


def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)

In [7]:
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

From arrays to tensors

In [8]:
import torch


x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

Tensors are defined by their contents: they are big rectangular blocks of numbers.

In [9]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


Indexing

In [10]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

In [11]:
#Num dimensions
x_train.ndim, y_train.ndim

(2, 1)

In [12]:
x_train[0, 0], y_train[0]

(tensor(0.), tensor(5))

In [13]:
# Shapes
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


### Random handwritten digits


In [14]:
# re-execute this cell for more samples
import random

import wandb  # just for some convenience methods that convert tensors to human-friendly datatypes

import text_recognizer.metadata.mnist as metadata # metadata module holds metadata separate from data

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])  # the label of the image
wandb.Image(example.reshape(*metadata.DIMS)).image  # the image itself

tensor(0)


### From scratch model

In [15]:
import math
import torch


weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [16]:
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [17]:
def log_softmax(x: torch.Tensor) -> torch.Tensor:
    return x - torch.log(torch.sum(torch.exp(x), axis=1))[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return log_softmax(linear(xb))

In [18]:
bs = 64  # batch size

xb = x_train[0:bs]  # a batch of inputs
outs = model(xb)  # outputs on that batch

print(outs[0], outs.shape)  # outputs on the first element of the batch

tensor([-2.5763, -1.7125, -2.1168, -2.0477, -2.6708, -2.7564, -2.2827, -2.5472,
        -2.1951, -2.6634], grad_fn=<SelectBackward0>) torch.Size([64, 10])


In [19]:
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [20]:
yb = y_train[0:bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.1094)


In [21]:
try:
    acc.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [22]:
def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -output[range(target.shape[0]), target].mean()

loss_func = cross_entropy

In [23]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.3123, grad_fn=<NegBackward0>) tensor(2.3026)


In [24]:
loss = loss_func(outs, yb)

loss.backward()

In [25]:
bias.grad

tensor([-0.0170, -0.0144,  0.0529, -0.0093, -0.0362,  0.0252,  0.0349,  0.0198,
        -0.0055, -0.0505])

In [26]:
lr = 0.5  # learning rate hyperparameter
epochs = 2  # how many epochs to train for

for epoch in range(epochs):  # loop over the data repeatedly
    for ii in range((n - 1) // bs + 1):  # in batches of size bs, so roughly n / bs of them
        start_idx = ii * bs  # we are ii batches in, each of size bs
        end_idx = start_idx + bs  # and we want the next bs entires

        # pull batches from x and from y
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # run model
        pred = model(xb)

        # get loss
        loss = loss_func(pred, yb)

        # calculate the gradients with a backwards pass
        loss.backward()

        # update the parameters
        with torch.no_grad():  # we don't want to track gradients through this part!
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # ACHTUNG: PyTorch doesn't assume you're done with gradients
            #          until you say so -- by explicitly "deleting" them,
            #          i.e. setting the gradients to 0.
            weights.grad.zero_()
            bias.grad.zero_()

In [27]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0816, grad_fn=<NegBackward0>) tensor(1.)


In [28]:
# re-execute this cell for more samples
idx = random.randint(0, len(x_train))
example = x_train[idx:idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(3)


### torch.nn components

In [29]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [30]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))  # should be unchanged from above!

tensor(0.0816, grad_fn=<NllLossBackward0>) tensor(1.)


Classy

In [31]:
from torch import nn

class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()  # the nn.Module.__init__ method does import setup, so this is mandatory
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

In [32]:
def forward(self, xb: torch.Tensor) -> torch.Tensor:
    return xb @ self.weights + self.bias

MNISTLogistic.forward = forward

model = MNISTLogistic()  # instantiated as an object
print(model(xb)[:4])  # callable like a function
loss = loss_func(model(xb), yb)  # composable like a function
loss.backward()  # we can still take gradients through it
print(model.weights.grad[::17,::2])  # and they show up in the .grad attribute

tensor([[-0.0580,  0.1888, -0.0354, -0.5416,  0.0383, -0.5461,  0.1413, -0.9100,
          0.1955, -0.2180],
        [-0.5383, -0.2115, -0.2720, -0.7730, -0.1205, -0.2894,  0.1344, -0.3800,
         -0.3505,  0.3573],
        [-0.4333,  0.1738, -0.2802, -0.0995, -0.0609, -0.4295,  0.2541, -0.5983,
          0.1048, -0.0539],
        [-0.5309, -0.2772, -0.2097, -0.3301, -0.1478, -0.1996,  0.2732, -0.6092,
         -0.1398,  0.3469]], grad_fn=<SliceBackward0>)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0056,  0.0082,  0.0088,  0.0080,  0.0077],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0374, -0.0024,  0.0528,  0.057

In [33]:

print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0172, -0.0556,  0.0517,  ..., -0.0461, -0.0101, -0.0111],
        [-0.0247,  0.0041,  0.0099,  ...,  0.0022,  0.0179, -0.0108],
        [ 0.0060,  0.0657, -0.0154,  ...,  0.0095,  0.0129, -0.0090],
        ...,
        [-0.0244, -0.0197, -0.0175,  ..., -0.0078,  0.0224, -0.0744],
        [-0.0077, -0.0143,  0.0279,  ..., -0.0035, -0.0164,  0.0105],
        [ 0.0871, -0.0206, -0.0215,  ..., -0.0271,  0.0016,  0.0516]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [34]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs
            end_idx = start_idx + bs
            xb = x_train[start_idx:end_idx]
            yb = y_train[start_idx:end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():  # finds params automatically
                    p -= p.grad * lr
                model.zero_grad()

fit()

In [35]:
print(accuracy(model(xb), yb))

tensor(1.)


## More refactoring

In [36]:
import textwrap

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")

torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [37]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)  # pytorch finds the nn.Parameters inside this nn.Module

    def forward(self, xb):
        return self.lin(xb)  # call nn.Linear.forward here

In [38]:
model = MNISTLogistic()
print(loss_func(model(xb), yb))  # loss is still close to 2.3

tensor(2.3471, grad_fn=<NllLossBackward0>)


In [39]:
print(*list(model.children()))

Linear(in_features=784, out_features=10, bias=True)


In [40]:
list(model.children())

[Linear(in_features=784, out_features=10, bias=True)]

In [41]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0192, -0.0186,  0.0115,  ...,  0.0354, -0.0024, -0.0329],
        [ 0.0021,  0.0258,  0.0182,  ..., -0.0221, -0.0090,  0.0193],
        [ 0.0167, -0.0221,  0.0237,  ...,  0.0076,  0.0221, -0.0029],
        ...,
        [-0.0296,  0.0267, -0.0296,  ..., -0.0333, -0.0013, -0.0033],
        [-0.0203,  0.0292, -0.0109,  ..., -0.0344,  0.0138, -0.0261],
        [ 0.0023, -0.0327,  0.0296,  ..., -0.0318, -0.0311, -0.0213]],
       requires_grad=True)
Parameter containing:
tensor([-0.0024, -0.0078, -0.0356,  0.0324,  0.0089, -0.0323, -0.0211, -0.0103,
         0.0016,  0.0103], requires_grad=True)


In [42]:
list(model.parameters())[0]

Parameter containing:
tensor([[ 0.0192, -0.0186,  0.0115,  ...,  0.0354, -0.0024, -0.0329],
        [ 0.0021,  0.0258,  0.0182,  ..., -0.0221, -0.0090,  0.0193],
        [ 0.0167, -0.0221,  0.0237,  ...,  0.0076,  0.0221, -0.0029],
        ...,
        [-0.0296,  0.0267, -0.0296,  ..., -0.0333, -0.0013, -0.0033],
        [-0.0203,  0.0292, -0.0109,  ..., -0.0344,  0.0138, -0.0261],
        [ 0.0023, -0.0327,  0.0296,  ..., -0.0318, -0.0311, -0.0213]],
       requires_grad=True)

### Optimizer

In [43]:
from torch import optim

def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [44]:
model = MNISTLogistic()
opt = configure_optimizer(model)

print("before training:", loss_func(model(xb), yb), sep="\n\t")

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print("after training:", loss_func(model(xb), yb), sep="\n\t")

before training:
	tensor(2.4095, grad_fn=<NllLossBackward0>)
after training:
	tensor(0.8542, grad_fn=<NllLossBackward0>)


### Dataset

In [45]:
from text_recognizer.data.util import BaseDataset

train_ds = BaseDataset(x_train, y_train)

In [46]:
BaseDataset??

In [47]:
model = MNISTLogistic()
opt = configure_optimizer(model)

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]  # xb and yb in one line!
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.8333, grad_fn=<NllLossBackward0>)


### Data Loader

In [48]:
from torch.utils.data import DataLoader

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [49]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimizer(self)

    for epoch in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit

In [50]:
odel = MNISTLogistic()

model.fit(train_dataloader)

print(loss_func(model(xb), yb))

tensor(0.6764, grad_fn=<NllLossBackward0>)


### Different Model... MLP

In [51]:
from text_recognizer.models.mlp import MLP

MLP.fit = fit  # attach our fitting loop

In [52]:
MLP.__init__??

In [53]:
digits_to_9 = list(range(10))

data_config = {"input_dims": (784,), "mapping": {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [54]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [55]:
model.fc1.weight.shape

torch.Size([1024, 784])

In [56]:
%%time

print("before training:", loss_func(model(xb), yb))

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader)

print("after training:", loss_func(model(xb), yb))

before training: tensor(2.3112, grad_fn=<NllLossBackward0>)
after training: tensor(0.2160, grad_fn=<NllLossBackward0>)
CPU times: user 19.5 s, sys: 38.2 ms, total: 19.6 s
Wall time: 9.78 s


### GPU

In [57]:
if torch.cuda.is_available():
    !nvidia-smi
else:
    print("☹️")

Sun Aug 21 02:43:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    14W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [58]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

loss_func(model(xb.to(device)), yb.to(device))

tensor(0.4052, device='cuda:0', grad_fn=<NllLossBackward0>)

In [59]:
def push_to_device(tensor):
    return tensor.to(device)

train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [60]:
%%time

model = MLP(data_config)
model.to(device)

model.fit(train_dataloader)

print(loss_func(model(push_to_device(xb)), push_to_device(yb)))

tensor(0.3047, device='cuda:0', grad_fn=<NllLossBackward0>)
CPU times: user 5.73 s, sys: 299 ms, total: 6.03 s
Wall time: 6.06 s


## Data validation

In [61]:
class MNISTDataModule:
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"
    
    def __init__(self, dir, bs=32):
        self.dir = dir
        self.bs = bs
        self.path = self.dir / self.filename

    def prepare_data(self):
        if not (self.path).exists():
            content = requests.get(self.url + self.filename).content
            self.path.open("wb").write(content)

    def setup(self):
        with gzip.open(self.path, "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

        x_train, y_train, x_valid, y_valid = map(
            torch.tensor, (x_train, y_train, x_valid, y_valid)
            )
        
        self.train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
        self.valid_ds = BaseDataset(x_valid, y_valid, transform=push_to_device, target_transform=push_to_device)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.bs, shuffle=True)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.valid_ds, batch_size=2 * self.bs, shuffle=False)

In [62]:
def fit(self: nn.Module, datamodule):
    datamodule.prepare_data()
    datamodule.setup()

    val_dataloader = datamodule.val_dataloader()
    
    self.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

    print("before start of training:", valid_loss / len(val_dataloader))

    opt = configure_optimizer(self)
    train_dataloader = datamodule.train_dataloader()
    for epoch in range(epochs):
        self.train()
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

        self.eval()
        with torch.no_grad():
            valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

        print(epoch, valid_loss / len(val_dataloader))


MNISTLogistic.fit = fit
MLP.fit = fit

In [69]:
model = MLP(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=32)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3024, device='cuda:0')
0 tensor(0.1682, device='cuda:0')
1 tensor(0.1100, device='cuda:0')


In [70]:
val_dataloader = datamodule.val_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

tensor(0.9685, device='cuda:0')

## Exercise

In [71]:
%%time 
from argparse import Namespace  # you'll need this

#args = None  # edit this
args = {'fc1': 512, 'fc2': 256, 'fc_dropout': 0.25}
args = Namespace(**args)

epochs = 2  # used in fit
bs = 32  # used by the DataModule


# used in fit, play around with this if you'd like
def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)


model = MLP(data_config, args=args)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=bs)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3049, device='cuda:0')
0 tensor(0.1499, device='cuda:0')
1 tensor(0.1110, device='cuda:0')
CPU times: user 9.77 s, sys: 585 ms, total: 10.4 s
Wall time: 10.3 s


In [72]:
val_dataloader = datamodule.val_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

tensor(0.9664, device='cuda:0')